In [1]:
import os
import pandas as pd
while os.getcwd() != '/home/jupyter/crisp':
    os.chdir("..")
%pwd

'/home/jupyter/crisp'

In [2]:
from models.MouseInitialisedSimpleNonLinearInvariantRiskMinimization import *
from models.SimpleNonLinearInvariantRiskMinimization import SimpleNonLinearInvariantRiskMinimization 
from dataio.DataFrameDataset import *
from dataio.datasets import *

In [3]:
from synthetic.facebook_synthetic_data_generator import generator_example

dim_inv = 2
dim_spu = 2
n_exp_train = 6 #int(2e2)
n_exp_test = 6
n_env = 3
save_dir = 'data/synthetic'

In [4]:
config = {
    "name": "Example Experiment for AH casual ensemble",
    "short_name": "ah_experiment_notebook",
    "bucket_project": "fdl-us-astronaut-health",
    "bucket_name": "ah_21_data",
    "bucket_path": "gs://ah_21_data",
    "verbose": 1,
    "test_val_split": [0.1, 0.1],
    "per_variant_experiment": False,
    "data_options": {
        #'dataset_fp': '../data/test_multiclass.pkl',
        'dataset_fp' : None,
        'output_data_regime' : ["real-valued", "binary", "binary", "real-valued", "multi-class"],
        'subject_keys': 'Subj_ID',
        'targets': ['Target'],
 #        'predictors': ['All'],
        'predictors': None,
        'environments': ['env_split'],
        'exclude': ['Subj_ID'],
        'synthetic_train_test_split' : True,
    },
    "feature_selection_options": {
        "max_features": 20,
        "verbose": 0,
        "seed": 12
    },
    "ensemble_options": {
        "models": ["ERM", "RF", "ICP", "IRM", "DCF", "ITE", "LIRM", "NLICP"]
    },
    "use_cloud":False,
    "results_directory": "results/"
}
data_config = config['data_options']

In [5]:
LIRM_options = config.get("ensemble_options").get('LIRM', {})
LIRM_args = {
            "use_icp_initialization": False,
            "verbose": 1,
            "n_iterations": 100, # 1000
            "seed": 0,
            "lr": 0.005, # 0.001
            "cuda": False
        }
LIRM_args.update(LIRM_options)

In [6]:
def get_dataset_for_synthetic_experiment(n_example, dim_inv, dim_spu, n_exp_train, n_exp_test, n_env, save_dir):
    # training data
    generator_example(n_example, dim_inv, dim_spu, n_exp_train, n_env ,save_dir, False)
    outfile_train = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp_train, n_env, False))

    # test data
    generator_example(n_example, dim_inv, dim_spu, n_exp_test, n_env ,save_dir, True)
    outfile_test = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp_test, n_env, True))
    
    return outfile_train, outfile_test

In [7]:
results = {}

for n_example in [1]:#,2,3,4,5]:
    output_data_regime = config["data_options"]["output_data_regime"][n_example-1]
    print("\n\nn_example:", n_example, "output_data_regime:", output_data_regime, "\n\n")
    
    outfile_train, outfile_test = get_dataset_for_synthetic_experiment(n_example, dim_inv, dim_spu, n_exp_train, n_exp_test, n_env, save_dir)
    df_train = pd.read_pickle(outfile_train)
    df_test = pd.read_pickle(outfile_test)

    config["data_options"]["dataset_fp_train"] = outfile_train
    config["data_options"]["dataset_fp_test"] = outfile_test
    config["data_options"]["predictors"] = list(df_train.columns[0:dim_inv+dim_spu])
    environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

    LIRM_args["target"] = config["data_options"]["targets"]
    LIRM_args["output_data_regime"] = output_data_regime
    LIRM_args["columns"] = test_dataset.predictor_columns

    mouse_base_data = MouseInitialiser(environment_datasets, val_dataset, test_dataset, LIRM_args)



n_example: 1 output_data_regime: real-valued 


Environments variables: {'E0': 0.1, 'E1': 1.5, 'E2': 2}
Generated Synthetic Data according to the Facebook setup Example: 1
     df with  4  columns
Environments variables: {'E0': 0.1, 'E1': 1.5, 'E2': 2}
Generated Synthetic Data according to the Facebook setup Example: 1
     df with  4  columns
Running a per sample experiment
Using synthetic dataset train/test split
Loaded  3  train environments
Env  0  has  2  samples
X shape  (2, 4)  y shape  (2, 1)
Env  1  has  2  samples
X shape  (2, 4)  y shape  (2, 1)
Env  2  has  2  samples
X shape  (2, 4)  y shape  (2, 1)
Loaded test set, X shape: (6, 4)  y shape:  (6, 1)
Start simple nonlinear-IRM training procedure
Sequential(
  (0): Linear(in_features=5, out_features=5, bias=False)
  (1): Sigmoid()
  (2): Linear(in_features=5, out_features=5, bias=False)
)
IRM (reg=0.001) has 21.216 validation error.


In [8]:
mouse_pre_load = MouseInitialisedSimpleNonLinearInvariantRiskMinimization(environment_datasets,
                                                                          val_dataset,
                                                                          test_dataset,
                                                                          LIRM_args,
                                                                          mouse_base_data.get_phi_params)

Start simple nonlinear-IRM training procedure
could not assign weight from Sigmoid() to Sigmoid()
IRM (reg=0.001) has 20.944 validation error.


In [19]:
mouse_base_data.get_phi_params

In [49]:
torch.nn.Parameter(ass[0].weight.detach())

Parameter containing:
tensor([[ 0.0068,  0.2296, -0.3580, -0.3190, -0.1621],
        [ 0.1101,  0.0010,  0.3448, -0.0495,  0.1085],
        [-0.1403, -0.0879, -0.4307, -0.2973, -0.1829],
        [ 0.0266,  0.1666,  0.2784, -0.2931, -0.1847],
        [ 0.1725,  0.3612, -0.0819,  0.3448, -0.0620]], requires_grad=True)

In [52]:
for x,y  in zip(ass, mouse_base_data.phi):
    try: 
        with torch.no_grad():
            y.weight = torch.nn.Parameter(x.weight.detach())
    except:
        print('could not assign weight from '+str(x)+' to '+str(y))

could not assign weight from Sigmoid() to Sigmoid()


In [23]:
[x for x in mouse_base_data.phi]

[Linear(in_features=5, out_features=5, bias=False),
 Sigmoid(),
 Linear(in_features=5, out_features=5, bias=False)]

In [18]:
for param in mouse_base_data.phi.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0068,  0.2296, -0.3580, -0.3190, -0.1621],
        [ 0.1101,  0.0010,  0.3448, -0.0495,  0.1085],
        [-0.1403, -0.0879, -0.4307, -0.2973, -0.1829],
        [ 0.0266,  0.1666,  0.2784, -0.2931, -0.1847],
        [ 0.1725,  0.3612, -0.0819,  0.3448, -0.0620]], requires_grad=True)
Parameter containing:
tensor([[ 0.0373,  0.3950, -0.4248, -0.2915, -0.1232],
        [-0.1843,  0.3765, -0.2998, -0.2159, -0.3224],
        [-0.4288, -0.2710,  0.3745,  0.1896,  0.2068],
        [ 0.0135, -0.2392,  0.0657, -0.4275, -0.3331],
        [-0.2405,  0.2722,  0.2523, -0.2083, -0.0261]], requires_grad=True)


In [11]:
mouse_base_data.get_phi_params[1]

Sigmoid()

In [12]:
mouse_base_data.get_phi_params[2]

Linear(in_features=5, out_features=5, bias=False)

In [8]:
mouse_base_data.phi[0]

NameError: name 'mouse_base_data' is not defined

In [33]:
mouse_base_data.phi.parameters

AttributeError: 'Tensor' object has no attribute 'parameters'

In [26]:
import torch
dim_x = 10

phi = torch.nn.Sequential(torch.nn.Linear(dim_x, dim_x, bias=False), torch.nn.Sigmoid(), torch.nn.Linear(dim_x, dim_x, bias=False))

In [40]:
phi[0]

Linear(in_features=10, out_features=10, bias=False)

In [39]:
mouse_base_data.phi[0]

tensor([ 0.7885, -0.2382, -0.2136, -0.2192, -0.1973], grad_fn=<SelectBackward>)

In [9]:
ass = MouseInitialisedLinearInvariantRiskMinimization(environment_datasets, val_dataset, test_dataset, LIRM_args, initial_weights)

Start IRM training procedure with real-valued target
iteration: 0 error: 17.620630264282227
IRM (reg=0.000) has 5.344 validation error.
	new best model:
	error: 5.3438720703125
	reg: 0
	phi.trace: tensor(3.1656, grad_fn=<TraceBackward>)
iteration: 0 error: 8.01580810546875
IRM (reg=0.000) has 5.176 validation error.
	new best model:
	error: 5.175507068634033
	reg: 1e-05
	phi.trace: tensor(2.9480, grad_fn=<TraceBackward>)
iteration: 0 error: 7.763260841369629
IRM (reg=0.000) has 5.074 validation error.
	new best model:
	error: 5.073523044586182
	reg: 0.0001
	phi.trace: tensor(2.9337, grad_fn=<TraceBackward>)
iteration: 0 error: 7.610284805297852
IRM (reg=0.001) has 5.004 validation error.
	new best model:
	error: 5.004339694976807
	reg: 0.001
	phi.trace: tensor(2.9294, grad_fn=<TraceBackward>)
iteration: 0 error: 7.506511688232422
IRM (reg=0.010) has 4.594 validation error.
	new best model:
	error: 4.593980312347412
	reg: 0.01
	phi.trace: tensor(2.9235, grad_fn=<TraceBackward>)
iteratio

In [21]:
import torch

False

In [10]:
lirm.w
# I think the test logits are the things you need to mouse-initialise here.

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]], requires_grad=True)

tensor([[ 0.7885, -0.2382, -0.2136, -0.2192, -0.1973],
        [ 0.0844,  0.8118, -0.0023,  0.0131,  0.1613],
        [-0.2093, -0.2198,  0.7903, -0.2175, -0.1949],
        [-0.2116, -0.2398, -0.2137,  0.7812, -0.1965],
        [-0.1837, -0.1961, -0.1864, -0.1901,  0.7792]],
       grad_fn=<CloneBackward>)

In [49]:
print(results.get(n_example).keys())
"acc/r2:", results.get(n_example).get("test_acc")

dict_keys(['test_logits', 'test_acc', 'test_nll', 'test_probs', 'test_labels', 'feature_coeffients', 'to_bucket'])


('acc/r2:', tensor(0.2576))

In [51]:
result = results.get(n_example).get("to_bucket")

if output_data_regime == "multi-class":
    r = sorted(zip(result.get("features"), np.abs(result.get("coefficients")).sum(axis=1)), key=lambda x: x[1], reverse=True)
else:
    r = sorted(zip(result.get("features"), result.get("coefficients")), key=lambda x: abs(x[1]), reverse=True)
r

[('Causal_2', 28.2161705493927),
 ('Causal_1', 26.71197199821472),
 ('Causal_0', 26.41683316230774),
 ('Causal_3', 21.73872461915016),
 ('Non_causal_7', 2.7639010846614838),
 ('Non_causal_5', 2.7266255617141724),
 ('Non_causal_2', 2.3941598534584045),
 ('Non_causal_8', 2.0637218952178955),
 ('Non_causal_6', 1.5529405996203423),
 ('Non_causal_4', 1.2595409452915192),
 ('Non_causal_3', 1.1885599195957184),
 ('Non_causal_9', 0.9855049755424261),
 ('Non_causal_1', 0.6325856596231461),
 ('Non_causal_0', 0.30464520677924156)]